In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim

import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [19]:
class z24Dataset(Dataset):
    def __init__(self, window_size=100, skipstep=50, lookahead=1, include_environmental_data=False):
        self.window_size = window_size
        self.skipstep = skipstep
        self.lookahead = lookahead
        self.include_environmental_data = include_environmental_data        
        self.index_file = np.loadtxt('../data/z24/name_to_index_zipped.txt',dtype=str)
        self.name_index_dict = dict(zip(range(len(self.index_file)),list(self.index_file)))
        self.slices_per_file = (65536-(self.window_size+self.lookahead))//self.skipstep
        
    def __len__(self):
        return len(self.index_file) * self.slices_per_file

    def __getitem__(self, index):
        index_to_read = index // self.slices_per_file
        file_to_read = self.name_index_dict[index_to_read]
        index_in_dataframe = index - index_to_read*self.slices_per_file
        
        data = np.load('../data/z24/permanent_npy/'+file_to_read+'.npy')
        
        X = data[index_in_dataframe:index_in_dataframe+self.window_size,:].flatten()
        Y = data[index_in_dataframe+self.window_size+self.lookahead,:].flatten() 
        
        return X,Y

In [20]:
# Neural Network Model

class Model(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = F.dropout(x, p=0.01, training=True)
        x = self.fc1(x)
        x = F.leaky_relu(x)
        x = F.dropout(x, p=0.01, training=True)
        x = self.fc2(x)
        return x

In [21]:
test = z24Dataset(window_size=100, skipstep=50, lookahead=1)

dataloader = DataLoader(test, batch_size=64, shuffle=True, num_workers=4)

model = Model(input_size=8*100, hidden_size=50, output_size=8)

optimizer = torch.optim.SGD(model.parameters(),
                            lr=0.01,
                            weight_decay=1.0778889384649698e-06,
                            momentum=0.9)

loss_criterion = torch.nn.MSELoss()

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [22]:
train_loss = []

for epoch in range(10):
    print(f'Epoch {epoch}/40')
    batchloss_train = []
    
    for X_train, Y_train in dataloader:  
        X_train_tensor = Variable(X_train).float()
        Y_train_tensor = Variable(Y_train).float()

        optimizer.zero_grad()  # zero the gradient buffer
        predicted_Y = model(X_train_tensor)
        trainloss = loss_criterion(predicted_Y, Y_train_tensor)
        trainloss.backward()
        optimizer.step()
    
        batchloss_train.append(trainloss.data[0])
        
    scheduler.step()
    train_loss.append(np.mean(batchloss_train))

Epoch 0/40
Epoch 1/40


Process Process-18:
Process Process-17:
Process Process-20:
Process Process-19:


KeyboardInterrupt: 

Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/dsteinar/miniconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/dsteinar/miniconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/dsteinar/miniconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/home/dsteinar/miniconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/dsteinar/miniconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 42, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/dsteinar/miniconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 42, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/dsteina

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan